## SIMPLE CHAR-RNN 

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(0)  
print ("TENSORFLOW VERSION IS %s" % (tf.__version__))

TENSORFLOW VERSION IS 1.0.1


## DEFINE TRAINING SEQUENCE

In [2]:
quote1 = ("If you want to build a ship, "
          "don't drum up people to collect wood and don't assign them tasks and work,"
          " but rather teach them to long for the endless immensity of the sea.")
quote2 = ("Perfection is achieved, "
          "not when there is nothing more to add, "
          "but when there is nothing left to take away.")
sentence = quote2
print ("FOLLOWING IS OUR TRAINING SEQUENCE:")
print (sentence)

FOLLOWING IS OUR TRAINING SEQUENCE:
Perfection is achieved, not when there is nothing more to add, but when there is nothing left to take away.


## DEFINE VOCABULARY AND DICTIONARY

In [3]:
char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}
print ("VOCABULARY: ")
print (char_set)
print ("DICTIONARY: ")
print (char_dic)

VOCABULARY: 
[' ', ',', '.', 'P', 'a', 'c', 'b', 'e', 'd', 'g', 'f', 'i', 'h', 'k', 'm', 'l', 'o', 'n', 's', 'r', 'u', 't', 'w', 'v', 'y']
DICTIONARY: 
{' ': 0, ',': 1, '.': 2, 'P': 3, 'a': 4, 'c': 5, 'b': 6, 'e': 7, 'd': 8, 'g': 9, 'f': 10, 'i': 11, 'h': 12, 'k': 13, 'm': 14, 'l': 15, 'o': 16, 'n': 17, 's': 18, 'r': 19, 'u': 20, 't': 21, 'w': 22, 'v': 23, 'y': 24}


## CONFIGURE NETWORK

In [4]:
data_dim    = len(char_set)
num_classes = len(char_set)
hidden_size     = 64
sequence_length = 10  # Any arbitrary number

## SET TRAINING BATCHES

In [5]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index
    dataX.append(x)
    dataY.append(y)
    if i < 10:
        print ("[%4d/%4d] [%s]=>[%s]" % (i, len(sentence), x_str, y_str))
        print ("%s%s=>%s" % (' '*12, x, y))

[   0/ 107] [Perfection]=>[erfection ]
            [3, 7, 19, 10, 7, 5, 21, 11, 16, 17]=>[7, 19, 10, 7, 5, 21, 11, 16, 17, 0]
[   1/ 107] [erfection ]=>[rfection i]
            [7, 19, 10, 7, 5, 21, 11, 16, 17, 0]=>[19, 10, 7, 5, 21, 11, 16, 17, 0, 11]
[   2/ 107] [rfection i]=>[fection is]
            [19, 10, 7, 5, 21, 11, 16, 17, 0, 11]=>[10, 7, 5, 21, 11, 16, 17, 0, 11, 18]
[   3/ 107] [fection is]=>[ection is ]
            [10, 7, 5, 21, 11, 16, 17, 0, 11, 18]=>[7, 5, 21, 11, 16, 17, 0, 11, 18, 0]
[   4/ 107] [ection is ]=>[ction is a]
            [7, 5, 21, 11, 16, 17, 0, 11, 18, 0]=>[5, 21, 11, 16, 17, 0, 11, 18, 0, 4]
[   5/ 107] [ction is a]=>[tion is ac]
            [5, 21, 11, 16, 17, 0, 11, 18, 0, 4]=>[21, 11, 16, 17, 0, 11, 18, 0, 4, 5]
[   6/ 107] [tion is ac]=>[ion is ach]
            [21, 11, 16, 17, 0, 11, 18, 0, 4, 5]=>[11, 16, 17, 0, 11, 18, 0, 4, 5, 12]
[   7/ 107] [ion is ach]=>[on is achi]
            [11, 16, 17, 0, 11, 18, 0, 4, 5, 12]=>[16, 17, 0, 11, 18, 0, 4,

In [6]:
ndata      = len(dataX)
batch_size = 512
print ("     'NDATA' IS %d" % (ndata))
print ("'BATCH_SIZE' IS %d" % (batch_size))

     'NDATA' IS 97
'BATCH_SIZE' IS 512


## DEFINE PLACEHOLDERS

In [7]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])
X_OH = tf.one_hot(X, num_classes)
print ("'sequence_length' IS [%d]" % (sequence_length))
print ("    'num_classes' IS [%d]" % (num_classes))
print("'X' LOOKS LIKE \n   [%s]" % (X))  
print("'X_OH' LOOKS LIKE \n   [%s]" % (X_OH))

'sequence_length' IS [10]
    'num_classes' IS [25]
'X' LOOKS LIKE 
   [Tensor("Placeholder:0", shape=(?, 10), dtype=int32)]
'X_OH' LOOKS LIKE 
   [Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)]


## DEFINE MODEL

In [8]:
num_hidden = 128
with tf.variable_scope('CHAR-RNN', reuse=False):
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(X_OH, num_hidden, activation_fn=tf.nn.relu)
    _outputs, _states = tf.nn.dynamic_rnn(cell, _hiddens, dtype=tf.float32)
    _outputs  = tf.contrib.layers.fully_connected(_outputs, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs = tf.reshape(_outputs, [batch_size, sequence_length, num_classes])
print ("OUTPUTS LOOKS LIKE [%s]" % (outputs))
print ("MODEL DEFINED.")

OUTPUTS LOOKS LIKE [Tensor("CHAR-RNN/Reshape:0", shape=(512, 10, 25), dtype=float32)]
MODEL DEFINED.


## DEFINE TF FUNCTIONS

In [9]:
weights = tf.ones([batch_size, sequence_length]) # EQUAL WEIGHTS
seq_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss  = tf.reduce_mean(seq_loss)
optm  = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
print ("FUNCTIONS DEFINED.")

FUNCTIONS DEFINED.


## OPTIMIZE

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())
MAXITER = 2000
for i in range(MAXITER):
    randidx = np.random.randint(low=0, high=ndata, size=batch_size)
    batchX = [dataX[iii] for iii in randidx]
    batchY = [dataY[iii] for iii in randidx]
    feeds = {X: batchX, Y: batchY}
    _, loss_val, results = sess.run(
        [optm, loss, outputs], feed_dict=feeds)
    if (i%200) == 0:
        print ("[%5d] loss_val: %.5f " % (i, loss_val))

[    0] loss_val: 3.21864 
[  200] loss_val: 0.60841 
[  400] loss_val: 0.21809 
[  600] loss_val: 0.18507 
[  800] loss_val: 0.17569 
[ 1000] loss_val: 0.17517 
[ 1200] loss_val: 0.18346 
[ 1400] loss_val: 0.17449 
[ 1600] loss_val: 0.17073 
[ 1800] loss_val: 0.16770 


## PRINT CHARS

In [11]:
randidx = np.random.randint(low=0, high=ndata, size=batch_size)
batchX = [dataX[iii] for iii in randidx]
batchY = [dataY[iii] for iii in randidx]
feeds = {X: batchX, Y: batchY}
results = sess.run(outputs, feed_dict=feeds)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    chars = [char_set[t] for t in index]
    if j < 10:
        print (chars)

[' ', 't', 'h', 'i', 'n', 'g', ' ', 'm', 'e', 'f']
['e', ' ', 'i', 's', ' ', 'n', 'o', 't', 'h', 'i']
['e', 'e', 'c', 't', 'i', 'o', 'n', ' ', 'i', 's']
[' ', ',', ' ', 'n', 'o', 't', ' ', 'w', 'h', 'e']
['h', ' ', 'a', 'd', 'd', ',', ' ', 'b', 'u', 't']
[' ', ' ', 't', 'h', 'e', 'r', 'e', ' ', 'i', 's']
['s', 'g', ' ', 'm', 'e', 'f', 't', ' ', 't', 'o']
['t', 's', ' ', 'n', 'o', 't', 'h', 'i', 'n', 'g']
['h', 'i', 'r', 'e', ' ', 'i', 's', ' ', 'n', 'o']
['t', 'o', 'r', 'e', ' ', 't', 'o', ' ', 'a', 'd']


### SAMPLING FUNCTION 

In [12]:
LEN = 1;
# XL = tf.placeholder(tf.int32, [None, LEN])
XL     = tf.placeholder(tf.int32, [None, 1])
XL_OH  = tf.one_hot(XL, num_classes)
with tf.variable_scope('CHAR-RNN', reuse=True):
    cell_L = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    cell_L = rnn.MultiRNNCell([cell_L] * 2, state_is_tuple=True)
    istate = cell_L.zero_state(batch_size=1, dtype=tf.float32)
    # DYNAMIC RNN WITH FULLY CONNECTED LAYER
    _hiddens  = tf.contrib.layers.fully_connected(XL_OH, num_hidden, activation_fn=tf.nn.tanh)
    _outputs_L, states_L = tf.nn.dynamic_rnn(cell_L, _hiddens
                                , initial_state=istate, dtype=tf.float32)
    _outputs_L  = tf.contrib.layers.fully_connected(
        _outputs_L, num_classes, activation_fn=None)
    # RESHAPE FOR SEQUNCE LOSS
    outputs_L = tf.reshape(_outputs_L, [LEN, 1, num_classes])
print (XL)

def weighted_pick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))
def softmax(x):
    alpha = 1
    e_x = np.exp(alpha*(x - np.max(x)))
    return e_x / np.sum(e_x) # only difference

Tensor("Placeholder_2:0", shape=(?, 1), dtype=int32)


## SAMPLE

### BURNIN

In [13]:
prime = "Perfection is"
istateval = sess.run(cell_L.zero_state(1, tf.float32))
for c in prime[:-1]:
    index = char_dic[c]
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval

### SAMPLE

In [14]:
inval  = [[char_dic[prime[-1]]]]
outval, stateval = sess.run([outputs_L, states_L]
                    , feed_dict={XL:inval, istate:istateval})
istateval = stateval
index = np.argmax(outval)
char  = char_set[index]
chars = char
for i in range(200):
    inval = [[index]]
    outval, stateval = sess.run([outputs_L, states_L]
                        , feed_dict={XL:inval, istate:istateval})
    istateval = stateval
    # index = np.argmax(outval)
    index = weighted_pick(softmax(outval))
    char  = char_set[index]
    chars += char
print ("<SAMPLED SETENCE> \n %s" % (prime+chars))
print ("\n<ORIGINAL SENTENCE> \n %s" % (sentence))

<SAMPLED SETENCE> 
 Perfection is achieved, not when there is nothing left to take away. when there is nothing left to take away. add, but when there is nothing more to add, but when there is nothing left to take away. awhere add, but

<ORIGINAL SENTENCE> 
 Perfection is achieved, not when there is nothing more to add, but when there is nothing left to take away.
